In [1]:
from langame_client import LangameClient
from pprint import pprint
from firebase_admin import firestore
import requests
from bs4 import BeautifulSoup
import json
import openai

In [2]:
c = LangameClient()

In [29]:
# TODO: use protobuf
p = {}
p["type"] = "TopicGeneralist"
p["template"] = """This is a conversation starter that leads to extremely profound conversations between people about "[TOPIC]".
Starter: What do you think of the theory that animals have emotions?
Starter: What is the role of philosophy in human society?
Starter: How does one recognize truth and knowledge? What are the ways to acquire it?
Starter:"""
t = {
    "createdAt": firestore.SERVER_TIMESTAMP,
    "engine": {
        "parameters": {
            "model": "davinci",
            "temperature": 0.7,
            "maxTokens": 150,
            "topP": 1,
            "frequencyPenalty": 0.1,
            "presencePenalty": 0.1,
            "stop": ["\\n", "Starter:"],
        }
    }
}
p_col = langame_client._firestore_client.collection("prompts")
new_prompt_ref = p_col.add(p)
pprint(p_col.document(new_prompt_ref[1].id).collection("tags").add(t))

(DatetimeWithNanoseconds(2021, 7, 16, 15, 49, 26, 906587, tzinfo=datetime.timezone.utc),
 <google.cloud.firestore_v1.document.DocumentReference object at 0x7f4c13177610>)


In [22]:
p = {}
p["type"] = "TopicGeneralistFineTuned"
p["template"] = "[TOPIC]. ->"
t = {
    "createdAt": firestore.SERVER_TIMESTAMP,
    "engine": {
        "parameters": {
            "model": get_model_by_dataset_name("generate-generalist-0.0.1.jsonl")["fine_tuned_model"],
            "temperature": 0.7,
            "maxTokens": 150,
            "topP": 1,
            "frequencyPenalty": 0.1,
            "presencePenalty": 0.1,
            "stop": ["\\n", "Starter:"],
        }
    }
}
p_col = langame_client._firestore_client.collection("prompts")
new_prompt_ref = p_col.add(p)
pprint(p_col.document(new_prompt_ref[1].id).collection("tags").add(t))

(DatetimeWithNanoseconds(2021, 7, 30, 14, 29, 29, 759625, tzinfo=datetime.timezone.utc),
 <google.cloud.firestore_v1.document.DocumentReference object at 0x7f7db4610b80>)


In [14]:
for _ in range(10):
    langame_client.prompt_to_meme("knowledge", model="TopicGeneralist")

calling openai with {'id': '544wPT15TsrSuNtpAyUQ', 'prompt': 'This is a conversation starter that leads to extremely profound conversations between people about "knowledge".\nStarter: What do you think of the theory that animals have emotions?\nStarter: What is the role of philosophy in human society?\nStarter: How does one recognize truth and knowledge? What are the ways to acquire it?\nStarter:', 'parameters': {'topP': 1, 'temperature': 0.7, 'frequencyPenalty': 0.1, 'presencePenalty': 0.1, 'model': 'davinci', 'maxTokens': 150, 'stop': ['\\n', 'Starter:']}}
calling openai with {'id': '544wPT15TsrSuNtpAyUQ', 'prompt': 'This is a conversation starter that leads to extremely profound conversations between people about "knowledge".\nStarter: What do you think of the theory that animals have emotions?\nStarter: What is the role of philosophy in human society?\nStarter: How does one recognize truth and knowledge? What are the ways to acquire it?\nStarter:', 'parameters': {'temperature': 0.7

In [16]:
# TODO: use protobuf
p = {}
p["type"] = "IceBreakerGeneralist"
p["template"] = """This is conversation starters that leads to extremely personal discussions between people.
Starter: What is an interesting fact that few people know about you?
Starter: What is something that most people take for granted, but has a deeper meaning to you?
Starter:"""
t = {
    "createdAt": firestore.SERVER_TIMESTAMP,
    "engine": {
        "parameters": {
            "model": "davinci",
            "temperature": 0.7,
            "maxTokens": 150,
            "topP": 1,
            "frequencyPenalty": 0.1,
            "presencePenalty": 0.1,
            "stop": ["\\n", "Starter:"],
        }
    }
}
p_col = langame_client._firestore_client.collection("prompts")
new_prompt_ref = p_col.add(p)
pprint(p_col.document(new_prompt_ref[1].id).collection("tags").add(t))

(DatetimeWithNanoseconds(2021, 7, 16, 15, 34, 18, 122873, tzinfo=datetime.timezone.utc),
 <google.cloud.firestore_v1.document.DocumentReference object at 0x7f4c131a0fd0>)


In [28]:
langame_client.prompt_to_meme_ice_breaker()

calling openai with {'id': 'oCccJ82pVND4znQx3zM5', 'prompt': 'This is conversation starters that leads to extremely personal discussions between people.\nStarter: What is an interesting fact that few people know about you?\nStarter: What is something that most people take for granted, but has a deeper meaning to you?\nStarter:', 'parameters': {'temperature': 0.7, 'maxTokens': 150, 'frequencyPenalty': 0.1, 'model': 'davinci', 'presencePenalty': 0.1, 'stop': ['\\n', 'Starter:'], 'topP': 1}}


(DatetimeWithNanoseconds(2021, 7, 16, 15, 35, 59, 312111, tzinfo=datetime.timezone.utc),
 <google.cloud.firestore_v1.document.DocumentReference at 0x7f4c131a0dc0>)

In [18]:
topics = [t.id for t in langame_client._firestore_client.collection("topics").list_documents()]

In [21]:

completions = []
for t in langame_client._firestore_client.collection(u"memes")\
    .stream():
    completions.append({"topics": t.get("topics"), "meme": t.get("content")})
pprint(len(completions))


MsVDZJgZRowLLY1wtWuN
sOoNX5T6EgwzBabWY0Ee
440


In [27]:
dataset = []
for e in completions:
    ts = ", ".join(e["topics"])
    d = {"prompt": f"Topics:\n{ts}\nQuestion:", "completion": f" {e['meme']}\n###\n"}
    dataset.append(d)
len(dataset)

440

In [29]:
import json
file_name = 'data/generate-generalist-0.0.2.jsonl'
with open(file_name, 'w') as outfile:
    for entry in dataset:
        json.dump(entry, outfile)
        outfile.write('\n')

In [30]:
!openai tools fine_tunes.prepare_data -f $file_name

Analyzing...

- Your file contains 440 prompt-completion pairs
- There are 2 duplicated prompt-completion pairs. These are rows: [360, 377]
- All prompts end with suffix `\nQuestion:`
- All prompts start with prefix `Topics:
`
- All completions end with suffix `\n###\n`

Based on the analysis we will perform the following actions:
- [Recommended] Remove 2 duplicate rows [Y/n]: ^C



In [31]:
import openai
f = openai.File.create(
  file=open(file_name),
  purpose="fine-tune"
)
f_t = openai.FineTune.create(
    training_file=f["id"],
    model="curie"
)

Upload progress: 100%|██████████| 77.9k/77.9k [00:00<00:00, 194kit/s]


RateLimitError: You have reached the maximum number of fine-tunes allowed for your organization for this month (10). Please contact finetuning@openai.com and tell us about your use-case if you would like this limit increased.

In [15]:
model = get_model_by_dataset_name("generate-generalist-0.0.2.jsonl")
model

<FineTune fine-tune id=ft-5DIo5s9BIk7ZLwB8biEVnzHB at 0x7f7db4395e50> JSON: {
  "created_at": 1627653967,
  "fine_tuned_model": null,
  "hyperparams": {
    "batch_size": 4,
    "learning_rate_multiplier": 0.1,
    "n_epochs": 4,
    "prompt_loss_weight": 0.1,
    "use_packing": null
  },
  "id": "ft-5DIo5s9BIk7ZLwB8biEVnzHB",
  "model": "curie",
  "object": "fine-tune",
  "organization_id": "org-6iVquBDkjuvNsgZOGPJsaRAs",
  "result_files": [],
  "status": "pending",
  "training_files": [
    {
      "bytes": 35542,
      "created_at": 1627653912,
      "filename": "generate-generalist-0.0.1.jsonl",
      "id": "file-wgDEM5MCzKoqGfW8N5FCEL6U",
      "object": "file",
      "purpose": "fine-tune",
      "status": "processed",
      "status_details": null
    }
  ],
  "updated_at": 1627653972,
  "user_id": "user-fs8ZlgenluyrbB3JQwgRc7O8",
  "validation_files": []
}

In [76]:
openai.Completion.create(
  model=model["fine_tuned_model"],
  prompt="Transhumanism. ->",
  max_tokens=100,
  stop=["\n"]
)

<OpenAIObject text_completion id=cmpl-3O9em9S5iLE0b0IXIJSxFb004GtuE at 0x7f560072c770> JSON: {
  "choices": [
    {
      "finish_reason": "stop",
      "index": 0,
      "logprobs": null,
      "text": " Are there any changes in human evolution that we have not experienced yet in our lifetime, locations at which universal humanism is furthest from what we are used to, and perhaps will be even further away from some day in the future"
    }
  ],
  "created": 1626886004,
  "id": "cmpl-3O9em9S5iLE0b0IXIJSxFb004GtuE",
  "model": "curie:ft-beaumont-2021-07-21-16-19-54",
  "object": "text_completion"
}